Code trích rút đặc trưng HOG của tập 110 ảnh

- Lấy danh sách tên file trong thư mục ảnh

In [49]:
import os
import functools, math
dir_imgs = os.listdir('Children')

- Đọc danh sách ảnh

In [50]:
import cv2
import numpy as np
list_imgs = []

for i in list(dir_imgs):
    img = cv2.imread('Children/'+i, cv2.IMREAD_GRAYSCALE)
    list_imgs.append(img)

- Tính tích chập
    + Hàm compute(receptiveField, kernel): tính tích chập cho mỗi pixel với 2 tham số receptiveField(vùng được duyệt) và kernel(bộ lọc). Trả về giá trị tích chập
    + convolution(matrix, kernel): tính tích chập cho ma trận ảnh với 2 tham số matrix(ma trận ảnh) và kernel(bộ lọc). Trả về ma trận sau khi quét qua bộ lọc

In [51]:
def compute(receptiveField, kernel):
    result=0
    for i in range(len(kernel)):
        for j in range(len(kernel)):
            result += receptiveField[i][j]*kernel[i][j]
    return result

def convolution(matrix, kernel):
    featureMap = []
    for i in range(1,len(matrix)-1):
        x = []
        for j in range(1,len(matrix[0])-1):
            x.append(compute(matrix[i-1:i+2,j-1:j+2], kernel))
        featureMap.append(x)
    return featureMap

- Tính ma trận hướng Gradients và độ lớn Gradients cho từng ảnh
    + Hàm gradients(img): Trả về ma trận hướng gradients và độ lớn gradients với tham số img(ma trận ảnh)

In [52]:
def gradients(img):
    img = np.pad(img, pad_width=1)
    sobel_x = [[-1, 0, 1],
           [-2, 0, 2],
           [-1, 0, 1]]
    sobel_y = [[-1, -2, -1],
            [0, 0, 0],
            [1, 2, 1]]
    Gx = convolution(img, sobel_x)
    Gy = convolution(img, sobel_y)
    direction, magnitude = [], []
    for i in range(len(Gx)):
        d,m = [], []
        for j in range(len(Gx[0])):
            m.append(round(math.sqrt(Gx[i][j]**2+Gy[i][j]**2)))
            d.append(round(math.atan(Gy[i][j]/Gx[i][j])*180/math.pi) if Gx[i][j]!=0 else 90)
        direction.append(d)
        magnitude.append(m)
    return np.array(direction), np.array(magnitude)

- Tạo histogram cho ảnh
    + Hàm unitHistogram(direction, magnitude): Trả về vector histogram cho vùng có kích thước 8x8 pixel trong ảnh với 2 tham số direction(ma trận hướng gradients của vùng) và magnitude(ma trận độ lớn gradients của vùng)
    + Hàm histogramOfGradients(img): Trả về mảng các vector histogram đại diện cho từng vùng 8x8 trong ảnh với tham số img(ma trận ảnh) 

In [69]:
def unitHistogram(direction, magnitude):
    histogram = [0,0,0,0,0,0,0,0,0,0]
    for i in range(8):
        for j in range(8):
            x = direction[i][j]//20
            if direction[i][j]%20 == 0:
                histogram[x] += magnitude[i][j]
            else:
                k = round((direction[i][j]-x*20)*magnitude[i][j]/20)
                histogram[x] += magnitude[i][j]-k
                histogram[x+1] += k
    histogram[0] += histogram[-1]
    return histogram[:-1]

def histogramOfGradients(img):
    direction, magnitude = gradients(img)
    result = []
    for i in range(0,len(img),8):
        row = []
        for j in range(0, len(img[0]), 8):
            row.append(unitHistogram(direction[i:i+8,j:j+8], magnitude[i:i+8,j:j+8]))
        result.append(row)
    return np.array(result)

- Chuẩn hoá theo khối 16x16
    + Hàm norm2(arr): Chuẩn hoá vector theo chuẩn norm2 với tham số arr(vector cần chuẩn hoá)
    + Hàm normalize(matrix): Chuẩn hoá ảnh theo từng khối 16x16 với tham số matrix(ma trận cần chuẩn hoá) 

In [53]:
def norm2(arr):
    norm = 0
    for i in arr:
        norm += i**2
    if norm==0: return arr
    return np.round(np.divide(arr,math.sqrt(norm)), decimals=5)
    
def normalize(matrix):
    result = []
    for i in range(len(matrix)-1):
        row = []
        for j in range(len(matrix[0])-1):
            m = np.array(matrix[i:i+2,j:j+2]).flatten()
            row.append(norm2(m))
        result.append(row)
    return np.array(result)        

- Trích rút đặc trưng cho từng ảnh

In [55]:
train = []
for i in list_imgs:
    his = histogramOfGradients(i)
    his_norm = normalize(his)
    hog = his_norm.flatten()
    train.append(hog)
train

[array([0.53655, 0.     , 0.06102, ..., 0.     , 0.     , 0.     ]),
 array([0.53655, 0.     , 0.06102, ..., 0.     , 0.     , 0.     ]),
 array([0.53655, 0.     , 0.06102, ..., 0.1324 , 0.1011 , 0.16331]),
 array([0.53655, 0.     , 0.06102, ..., 0.09342, 0.06678, 0.00524]),
 array([0.48514, 0.0965 , 0.22645, ..., 0.12792, 0.02483, 0.01473]),
 array([0.53655, 0.     , 0.06102, ..., 0.24568, 0.14331, 0.05882]),
 array([0.53655, 0.     , 0.06102, ..., 0.16963, 0.07349, 0.0255 ]),
 array([0.53655, 0.     , 0.06102, ..., 0.08795, 0.04697, 0.004  ]),
 array([0.54216, 0.     , 0.06166, ..., 0.011  , 0.02153, 0.03732]),
 array([0.53655, 0.     , 0.06102, ..., 0.00544, 0.00435, 0.00664]),
 array([0.53655, 0.     , 0.06102, ..., 0.09037, 0.02621, 0.00971]),
 array([0.53655, 0.     , 0.06102, ..., 0.22547, 0.01057, 0.03269]),
 array([5.3655e-01, 0.0000e+00, 6.1020e-02, ..., 1.1700e-01, 6.9000e-04,
        4.6000e-04]),
 array([0.53655, 0.     , 0.06102, ..., 0.25389, 0.00887, 0.00707]),
 array([

- Tổ chức lưu trữ thông tin về ảnh

In [56]:
data = []
for i in range(len(train)):
    data.append({
        'features' : train[i].tolist(),
        'link' : dir_imgs[i]
    })
data

[{'features': [0.53655,
   0.0,
   0.06102,
   0.02029,
   0.26828,
   0.26828,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   8e-05,
   0.0,
   0.3066,
   0.3066,
   0.0,
   0.0,
   0.0,
   0.6132,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.00113,
   0.00023,
   0.00068,
   0.0006,
   0.0,
   0.0,
   0.0,
   0.0,
   8e-05,
   0.0,
   0.0,
   0.00012,
   0.0,
   0.4999,
   0.4999,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.00025,
   0.00012,
   0.50002,
   0.50002,
   0.0,
   0.0,
   0.00012,
   0.00184,
   0.00037,
   0.0011,
   0.00098,
   0.0,
   0.0,
   0.0,
   0.0,
   0.00012,
   0.0049,
   0.00257,
   0.00441,
   0.00796,
   0.00527,
   0.00208,
   0.00368,
   0.00184,
   0.00172,
   0.0,
   0.0,
   0.00019,
   0.0001,
   0.39268,
   0.39268,
   0.0,
   0.0,
   0.0001,
   0.00106,
   0.00058,
   0.00087,
   0.00096,
   0.34986,
   0.3948,
   0.04907,
   0.00019,
   0.00077,
   0.00385,
   0.00202,
   0.00346,
   0.00625,
   0.00414,
   0.00164,
   0.00289,

- Lưu thành file json

In [57]:
import json
with open("children.json", "w") as fout:
    json.dump(data, fout)

- Đọc dữ liệu từ file json

In [58]:
import json
f = open('children.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Closing file
f.close()